In [1]:
#Import Packages
using Pkg
using DataFrames
using CSV
using StatsBase
using GLM
using Lathe
using MLBase
using ClassImbalance
using ScikitLearn

In [2]:
#Enable printing of 1000 columns
ENV["COLUMNS"] = 1000

1000

In [3]:
#Load our data
df = CSV.read("C:\\Users\\olgak\\Documents\\df.csv", DataFrame)
first(df,5)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
,Int64,String7,Float64,Int64,Int64,String3,String15,String7,Float64,String7,String15,Int64
1,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,N/A,never smoked,1
3,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
4,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
5,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24,never smoked,1


In [4]:
#See the names of our variables
names(df)

12-element Vector{Symbol}:
 :id
 :gender
 :age
 :hypertension
 :heart_disease
 :ever_married
 :work_type
 :Residence_type
 :avg_glucose_level
 :bmi
 :smoking_status
 :stroke

In [5]:
#Check for missing values
[count(ismissing,col) for col in eachcol(df)]

12-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [6]:
#Check variables' type 
eltype.(eachcol(df))

12-element Vector{DataType}:
 Int64
 String7
 Float64
 Int64
 Int64
 String3
 String15
 String7
 Float64
 String7
 String15
 Int64

In [7]:
bmi = replace(df.bmi, "N/A" => "0")

5110-element PooledArrays.PooledVector{String, UInt32, Vector{UInt32}}:
 "36.6"
 "0"
 "32.5"
 "34.4"
 "24"
 "29"
 "27.4"
 "22.8"
 "0"
 "24.2"
 "29.7"
 "36.8"
 "27.3"
 ⋮
 "17.5"
 "0"
 "28.3"
 "24.5"
 "21.7"
 "46.9"
 "18.6"
 "0"
 "40"
 "30.6"
 "25.6"
 "26.2"

In [8]:
df.bmi = parse.(Float64, bmi)

5110-element Vector{Float64}:
 36.6
  0.0
 32.5
 34.4
 24.0
 29.0
 27.4
 22.8
  0.0
 24.2
 29.7
 36.8
 27.3
  ⋮
 17.5
  0.0
 28.3
 24.5
 21.7
 46.9
 18.6
  0.0
 40.0
 30.6
 25.6
 26.2

In [9]:
nm = deleteat!(bmi, findall(bmi->bmi==0.0,df.bmi))

4909-element PooledArrays.PooledVector{String, UInt32, Vector{UInt32}}:
 "36.6"
 "32.5"
 "34.4"
 "24"
 "29"
 "27.4"
 "22.8"
 "24.2"
 "29.7"
 "36.8"
 "27.3"
 "28.2"
 "30.9"
 ⋮
 "28.2"
 "40.8"
 "17.5"
 "28.3"
 "24.5"
 "21.7"
 "46.9"
 "18.6"
 "40"
 "30.6"
 "25.6"
 "26.2"

In [10]:
nmm = parse.(Float64, nm)

4909-element Vector{Float64}:
 36.6
 32.5
 34.4
 24.0
 29.0
 27.4
 22.8
 24.2
 29.7
 36.8
 27.3
 28.2
 30.9
  ⋮
 28.2
 40.8
 17.5
 28.3
 24.5
 21.7
 46.9
 18.6
 40.0
 30.6
 25.6
 26.2

In [11]:
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
,Int64,String7,Float64,Int64,Int64,String3,String15,String7,Float64,Float64,String15,Int64
1,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,0.0,never smoked,1
3,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
4,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
5,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
6,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
7,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
8,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
9,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,0.0,Unknown,1


In [12]:
m = mean(nmm)

28.893236911794666

In [13]:
df.bmi = replace(df.bmi, 0.0 => m)

5110-element Vector{Float64}:
 36.6
 28.893236911794666
 32.5
 34.4
 24.0
 29.0
 27.4
 22.8
 28.893236911794666
 24.2
 29.7
 36.8
 27.3
  ⋮
 17.5
 28.893236911794666
 28.3
 24.5
 21.7
 46.9
 18.6
 28.893236911794666
 40.0
 30.6
 25.6
 26.2

In [14]:
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
,Int64,String7,Float64,Int64,Int64,String3,String15,String7,Float64,Float64,String15,Int64
1,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.8932,never smoked,1
3,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
4,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
5,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
6,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
7,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
8,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
9,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,28.8932,Unknown,1


In [15]:
@sk_import preprocessing: LabelEncoder

PyObject <class 'sklearn.preprocessing._label.LabelEncoder'>

In [16]:
labelencoder = LabelEncoder() 
categories = [2 6 7 8 11] 
for col in categories 
     df[col] = fit_transform!(labelencoder, df[col]) 
end

In [17]:
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Int64,Int64
1,9046,1,67.0,0,1,1,2,1,228.69,36.6,1,1
2,51676,0,61.0,0,0,1,3,0,202.21,28.8932,2,1
3,31112,1,80.0,0,1,1,2,0,105.92,32.5,2,1
4,60182,0,49.0,0,0,1,2,1,171.23,34.4,3,1
5,1665,0,79.0,1,0,1,3,0,174.12,24.0,2,1
6,56669,1,81.0,0,0,1,2,1,186.21,29.0,1,1
7,53882,1,74.0,1,1,1,2,0,70.09,27.4,2,1
8,10434,0,69.0,0,0,0,2,1,94.39,22.8,2,1
9,27419,0,59.0,0,0,1,2,0,76.15,28.8932,0,1


In [18]:
deletecols!(df, 1)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Int64,Int64
1,1,67.0,0,1,1,2,1,228.69,36.6,1,1
2,0,61.0,0,0,1,3,0,202.21,28.8932,2,1
3,1,80.0,0,1,1,2,0,105.92,32.5,2,1
4,0,49.0,0,0,1,2,1,171.23,34.4,3,1
5,0,79.0,1,0,1,3,0,174.12,24.0,2,1
6,1,81.0,0,0,1,2,1,186.21,29.0,1,1
7,1,74.0,1,1,1,2,0,70.09,27.4,2,1
8,0,69.0,0,0,0,2,1,94.39,22.8,2,1
9,0,59.0,0,0,1,2,0,76.15,28.8932,0,1


In [19]:
countmap(df.stroke)

Dict{Int64, Int64} with 2 entries:
  0 => 4861
  1 => 249

In [20]:
X2, y2 =smote(df[!,[:gender,:age ,:hypertension, :heart_disease, :ever_married, :work_type, :Residence_type, :avg_glucose_level, :bmi , :smoking_status]], df.stroke, k = 5, pct_under = 150, pct_over = 200)
df_balanced = X2
df_balanced.stroke = y2

1494-element Vector{Int64}:
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 1
 1
 1
 ⋮
 0
 1
 0
 1
 0
 1
 1
 0
 0
 0
 0
 0

In [21]:
df = df_balanced

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,0.0,70.0,0.0,0.0,1.0,2.0,1.0,221.58,47.5,2.0,1
2,0.0,53.0,0.0,0.0,1.0,0.0,1.0,64.17,41.5,2.0,1
3,0.0,21.0,0.0,0.0,0.0,2.0,0.0,85.86,35.4,0.0,0
4,1.0,9.0,0.0,0.0,0.0,4.0,0.0,94.59,20.0,0.0,0
5,0.0,70.0,0.0,1.0,1.0,2.0,0.0,191.999,27.7239,1.73808,1
6,1.0,1.64,0.0,0.0,0.0,4.0,1.0,115.12,21.1,0.0,0
7,1.0,22.0,0.0,0.0,0.0,2.0,0.0,86.53,20.8,2.0,0
8,0.0,79.0,0.0,0.0,1.0,2.0,1.0,97.73,21.5,3.0,1
9,1.0,51.0,0.0,0.0,1.0,3.0,1.0,75.73,30.7,2.0,0


In [22]:
#Count the classes
countmap(df.stroke)

Dict{Int64, Int64} with 2 entries:
  0 => 747
  1 => 747

In [23]:
#Train test split
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df,.70);

In [24]:
#Train logistic regression model
fm = @formula(stroke ~ gender + age + hypertension + heart_disease + ever_married + work_type + Residence_type + avg_glucose_level + bmi + smoking_status)
logit = glm(fm, train, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

stroke ~ 1 + gender + age + hypertension + heart_disease + ever_married + work_type + Residence_type + avg_glucose_level + bmi + smoking_status

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────
                         Coef.   Std. Error      z  Pr(>|z|)    Lower 95%    Upper 95%
──────────────────────────────────────────────────────────────────────────────────────
(Intercept)        -2.98868     0.312039     -9.58    <1e-21  -3.60026     -2.37709
gender             -0.0221287   0.0951571    -0.23    0.8161  -0.208633     0.164376
age                 0.0425268   0.00303608   14.01    <1e-43   0.0365762    0.0484774
hypertension        0.271924    0.122861      2.21    0.0269   0.0311206    0.512726
heart_disease       0.2891

In [25]:
using MLBase: predict
prediction = predict(logit,test)

445-element Vector{Union{Missing, Float64}}:
 0.033082840575404136
 0.7599346808717621
 0.9028275400138044
 0.9072693116538992
 0.8501714491378547
 0.7159593191477192
 0.7926351422521033
 0.6149955886402599
 0.3885472949076769
 0.044999925977320054
 0.782108710371664
 0.8215839530339542
 0.4776039098994415
 ⋮
 0.8781963972679641
 0.005750813264251333
 0.39001420945182597
 0.6949641636226551
 0.5719892221705558
 0.014897283601475683
 0.2606058796103663
 0.7628812547394352
 0.5109698005139846
 0.7707134996302052
 0.16292690304256313
 0.0109247957920621

In [26]:
#Convert probability score to class
prediction_class = [if x < 0.5 0 else 1 end for x in prediction];
prediction_df = DataFrame(y_actual = test.stroke, y_predicted = prediction_class, prob_predicted = prediction);
prediction_df.correctly_classified = prediction_df.y_actual .== prediction_df.y_predicted

445-element BitVector:
 1
 1
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 1
 ⋮
 1
 1
 1
 1
 0
 1
 1
 1
 0
 1
 1
 1

In [27]:
#Accuracy Score
accuracy = mean(prediction_df.correctly_classified)
print("Accuracy of the model is : ",accuracy)

Accuracy of the model is : 0.7932584269662921

In [28]:
#Confusion Matrix
confusion_matrix = MLBase.roc(prediction_df.y_actual, prediction_df.y_predicted)

ROCNums{Int64}
  p = 224
  n = 221
  tp = 188
  tn = 165
  fp = 56
  fn = 36
